### Testing model on test dataset to verify accuracy and recall rate

In [1]:
import joblib
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

In [2]:
best_model = joblib.load('Created_files/best_isolation_forest_model.pkl')
best_model

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['I_MEAS', 'TC_LD', 'TC_CMB',
                                                   'TC_CPS', 'PD2'])])),
                ('model',
                 IsolationForest(contamination=0.4, max_features=0.6,
                                 max_samples=0.8, n_estimators=300,
                                 random_state=42))])

### Read pickled dataframe

In [3]:
no_psu_with_fake_data_df_test = pd.read_pickle('Created_files/no_psu_with_fake_data_df_test.pkl')
no_psu_with_fake_data_df_test

,Date_Time,unit_names,I_MEAS,TC_LD,TC_CMB,TC_CPS,PD2,is_anomaly_truth
65507,2024-04-02 18:48:59.555,L04,7.00,27.00,25.25,27.76,0.21,1
414920,2024-04-02 18:52:29.085,L16,11.00,26.00,24.75,22.71,0.03,1
465080,2024-04-16 13:05:16.984,L16,0.01,25.50,25.00,22.71,0.03,1
25073,2024-04-16 14:02:05.735,L02,9.00,30.00,27.00,29.18,2.68,1
217240,2024-04-02 18:40:10.823,L09,7.00,28.00,24.75,28.35,1.27,1
...,...,...,...,...,...,...,...,...
550631,NaT,L09,8.98,30.68,27.22,29.09,0.68,-1
588002,NaT,L14,10.02,30.67,25.47,27.90,3.33,-1
39725,2024-04-02 18:39:53.985,L03,6.00,26.25,26.00,23.53,0.32,1
356238,2024-04-16 14:02:25.570,L14,11.01,31.50,26.75,30.71,3.75,1


### Sample pickled dataframe to act as new dataset

In [4]:
# Define the features used for prediction
features = ['I_MEAS', 'TC_LD', 'TC_CMB', 'TC_CPS', 'PD2']

# Extract the feature data
new_X = no_psu_with_fake_data_df_test[features]

# Preprocess the new data
new_X_preprocessed = best_model.named_steps['preprocessor'].transform(new_X)

# Predict anomaly labels with the best model
new_anomaly_labels = best_model.named_steps['model'].predict(new_X_preprocessed)

# Add the predictions to the new data
no_psu_with_fake_data_df_test['is_anomaly_pred'] = new_anomaly_labels

no_psu_with_fake_data_df_test['is_anomaly_pred'].value_counts()
no_psu_with_fake_data_df_test


,Date_Time,unit_names,I_MEAS,TC_LD,TC_CMB,TC_CPS,PD2,is_anomaly_truth,is_anomaly_pred
65507,2024-04-02 18:48:59.555,L04,7.00,27.00,25.25,27.76,0.21,1,1
414920,2024-04-02 18:52:29.085,L16,11.00,26.00,24.75,22.71,0.03,1,-1
465080,2024-04-16 13:05:16.984,L16,0.01,25.50,25.00,22.71,0.03,1,1
25073,2024-04-16 14:02:05.735,L02,9.00,30.00,27.00,29.18,2.68,1,1
217240,2024-04-02 18:40:10.823,L09,7.00,28.00,24.75,28.35,1.27,1,1
...,...,...,...,...,...,...,...,...,...
550631,NaT,L09,8.98,30.68,27.22,29.09,0.68,-1,-1
588002,NaT,L14,10.02,30.67,25.47,27.90,3.33,-1,-1
39725,2024-04-02 18:39:53.985,L03,6.00,26.25,26.00,23.53,0.32,1,-1
356238,2024-04-16 14:02:25.570,L14,11.01,31.50,26.75,30.71,3.75,1,1


### Confusion Matrix to examine accuracy and more metrics

In [5]:
truth_labels = no_psu_with_fake_data_df_test['is_anomaly_truth']
predicted_labels = no_psu_with_fake_data_df_test['is_anomaly_pred']

# Creating the confusion matrix with labels [1, -1] where 1 = no anomaly, -1 = anomaly
conf_matrix = confusion_matrix(truth_labels, predicted_labels, labels=[1, -1])

print("Confusion Matrix:")
print(conf_matrix)

# True Positives (TP) --> No issues, predict no issues
# False Positives (FP) --> Anomalies, predict no issues
# False Negatives (FN) --> No issues, predict anomalies
# True Negatives (TN) --> Anomalies, predict anomalies
TP = conf_matrix[0, 0]
FP = conf_matrix[1, 0]
FN = conf_matrix[0, 1]
TN = conf_matrix[1, 1]

FPR = FP / (FP + TN)
FNR = FN / (FN + TP)
accuracy = accuracy_score(truth_labels, predicted_labels)


classification_rep = classification_report(truth_labels, predicted_labels, target_names=['anomaly', 'no anomaly'])
print("\nFalse Positive Rate (FPR):", FPR)
print("False Negative Rate (FNR):", FNR)
print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_rep)


Confusion Matrix:
[[73589 23299]
 [  329 26054]]

False Positive Rate (FPR): 0.012470151233749005
False Negative Rate (FNR): 0.24047353645446287
Accuracy: 0.8083247479131345

Classification Report:
              precision    recall  f1-score   support

     anomaly       0.53      0.99      0.69     26383
  no anomaly       1.00      0.76      0.86     96888

    accuracy                           0.81    123271
   macro avg       0.76      0.87      0.77    123271
weighted avg       0.90      0.81      0.82    123271



#### Summary: The model shows a high overall accuracy of 80.83%, indicating its robustness in classifying both anomalies and non-anomalies.

- For Anomalies: 
1. It displays good precision, with 53% of the predicted anomalies being accurate. (TN / (TN+FN))
2. It displays excellent recall, with 99% of the actual anomalies being captured. (TN / (TN+FP))

- For Non-Anomalies: 
1. It displays excellent precision, with 100% of the predicted non-anomalies being accurate. (TP / (TP+FP))
2. It displays good recall, with 76% of the actual non-anomalies being captured. (TP / (TP+FN))